In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords 

from numpy import array 
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

In [3]:
df = pd.read_csv('reviews.csv')
df.isnull().values.any() # checks if data file have any null values
df.head()

,reviews,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [4]:
def preprocess(text):
  text = re.sub('<[^>]*>', '', text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
  text = re.sub('[\W]+', ' ', text.lower()) +\
      ' '.join(emoticons).replace('-', '')
  return text


In [31]:
X = []
sentences = list(df['reviews'])
for sen in sentences:
    X.append(preprocess(sen))
y = df['sentiment']

# Train( 80 ) Test( 20 ) Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Embedding Layer ( For converting textual data into numeric data )

* First layer in DL model in Keras

In [7]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
print(len(X_train[1]))
print(len(X_test[1]))

158
233


In [8]:
# padding process 
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [9]:
print(len(X_train[1]))
print(len(X_test[1]))

100
100


In [10]:
print(vocab_size) # number of unique words in dataset

94320


In [11]:
# using GloVe for creating feature matrix
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [12]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [13]:
len(embedding_matrix)

94320

# Text Classification with LSTM

In [15]:
from keras.layers import LSTM
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False))
model.add(LSTM(200, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  after removing the cwd from sys.path.


In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          9432000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               240800    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 9,673,001
Trainable params: 241,001
Non-trainable params: 9,432,000
_________________________________________________________________


In [17]:
model.fit(X_train, y_train, epochs=5, batch_size=64)

Epoch 1/5
40000/40000 [==============================] - 125s 3ms/step - loss: 0.5793 - acc: 0.6917
Epoch 2/5
40000/40000 [==============================] - 149s 4ms/step - loss: 0.4398 - acc: 0.7966
Epoch 3/5
40000/40000 [==============================] - 126s 3ms/step - loss: 0.4037 - acc: 0.8147
Epoch 4/5
40000/40000 [==============================] - 138s 3ms/step - loss: 0.3758 - acc: 0.8296
Epoch 5/5
40000/40000 [==============================] - 126s 3ms/step - loss: 0.3531 - acc: 0.8415


In [29]:
model.save("Senti.h5py")

In [28]:
x='is the rare movie that succeeds on almost every level, where each character, scene, costume, and joke firing on all cylinders to make a film worth repeated viewings.'

In [26]:
# X = []
# X.append(preprocess(x))

In [30]:
X = tokenizer.texts_to_sequences(X)

In [22]:
X=pad_sequences(X, padding='post', maxlen=maxlen)

In [23]:
model.predict_proba(X)

array([[0.6318903]], dtype=float32)

In [24]:
model.predict_classes(X)

array([[1]])

In [50]:
n=56
instance = X[n]
instance = tokenizer.texts_to_sequences(instance)

flat_list = []
for sublist in instance:
    for item in sublist:
        flat_list.append(item)

flat_list = [flat_list]

instance = pad_sequences(flat_list, padding='post', maxlen=maxlen)

In [51]:
model.predict_proba(instance)
model.predict_classes(instance)

array([[1]])

In [52]:
y[n]

1

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

C:\Users\rawll\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\rawll\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\rawll\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\rawll\AppData\Roaming\Python\Python36\site-

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9037772386696442346
]
